# Behavioral Cloning

In [2]:
import cv2 as cv

import time
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [3]:
pathdata = "../data/"
# data_dir = './data/mydata'
# data_csv = '/driving_log.csv'
# model_json = 'model_behavioral_cloning_augmentation.json'
# model_weights = 'model_behavioral_cloning_augmentation.h5'

cnames = ['centerIm', 'leftIm','rightIm','steer','throttle','brake','speed']
df = pd.read_csv(pathdata+"driving_log.csv", names=cnames)
df.head()

,centerIm,leftIm,rightIm,steer,throttle,brake,speed
0,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,0.0,0.0,0.0,20.87538
1,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,0.0,0.0,0.0,20.43664
2,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,0.0,0.0,0.0,20.26660
3,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,0.0,0.0,0.0,20.46741
4,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,C:\Users\YHU51\SelfDrivingCarNanodegree\1Compu...,0.0,0.0,0.0,20.30720


In [7]:
xCenter = df['centerIm']
xLeft  = df['leftIm']
xRight = df['rightIm']
y = df['steer']
#Y_train = Y_train.astype(np.float32)
y.shape

(9100,)

In [8]:
# impath = xCenter[0]
# img = cv.imread(impath)
images = []
for i, ipath in enumerate(xCenter):
    img = cv.imread(ipath)
    images.append(img)
#     print(i, img)

X = np.array(images)
y = np.array(y)
# plt.imshow(img)
print(X.shape)
print(y.shape)


(9100, 160, 320, 3)
(9100,)


In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=0)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)

(6097, 160, 320, 3)
(3003, 160, 320, 3)
(6097,)


In [ ]:
# Read center image


In [14]:
# Simple Network
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape = (160,320,3)))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam")

model.fit(X, y, validation_split=0.33, shuffle=True, epochs = 5)
model.save("model.h5")


C:\Users\YHU51\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 6096 samples, validate on 3004 samples
Epoch 1/5
6096/6096 [==============================] - 9s 1ms/step - loss: 9835935.0507 - val_loss: 5103.7747
Epoch 2/5
6096/6096 [==============================] - 9s 1ms/step - loss: 3936.3061 - val_loss: 2887.1119
Epoch 3/5
6096/6096 [==============================] - 9s 1ms/step - loss: 3109.3608 - val_loss: 2269.2988
Epoch 4/5
6096/6096 [==============================] - 9s 1ms/step - loss: 2213.5746 - val_loss: 1778.9950
Epoch 5/5
6096/6096 [==============================] - 9s 1ms/step - loss: 2071.6933 - val_loss: 2611.9960


In [13]:
help(Flatten)

Help on class Flatten in module keras.layers.core:

class Flatten(keras.engine.topology.Layer)
 |  Flattens the input. Does not affect the batch size.
 |  
 |  # Example
 |  
 |  ```python
 |      model = Sequential()
 |      model.add(Conv2D(64, 3, 3,
 |                       border_mode='same',
 |                       input_shape=(3, 32, 32)))
 |      # now: model.output_shape == (None, 64, 32, 32)
 |  
 |      model.add(Flatten())
 |      # now: model.output_shape == (None, 65536)
 |  ```
 |  
 |  Method resolution order:
 |      Flatten
 |      keras.engine.topology.Layer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  call(self, inputs)
 |      This is where the layer's logic lives.
 |      
 |      # Arguments
 |          inputs: Input tensor, or list/tuple of input tensors.
 |          **kwargs: Additional keyword arguments.
 |      
 |      # Returns
 |     